## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chokurdakh,RU,70.6333,147.9167,38.50,73,1,10.00,clear sky
1,1,Georgetown,MY,5.4112,100.3354,83.89,86,20,0.00,few clouds
2,2,Kaitangata,NZ,-46.2817,169.8464,49.93,59,100,3.83,moderate rain
3,3,Tasiilaq,GL,65.6145,-37.6368,39.60,86,55,5.64,broken clouds
4,4,Kodiak,US,57.7900,-152.4072,53.11,82,90,5.75,haze


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Georgetown,MY,5.4112,100.3354,83.89,86,20,0.00,few clouds
6,6,Puerto Ayora,EC,-0.7393,-90.3518,80.56,84,37,8.99,scattered clouds
8,8,Rikitea,PF,-23.1203,-134.9692,70.50,67,76,6.44,broken clouds
10,10,Roald,NO,62.5833,6.1333,72.00,82,96,10.36,overcast clouds
12,12,Lompoc,US,34.6391,-120.4579,82.02,72,40,3.00,scattered clouds
16,16,Hithadhoo,MV,-0.6000,73.0833,80.92,75,78,4.50,broken clouds
17,17,Saint-Francois,FR,46.4154,3.9054,71.04,65,0,4.61,clear sky
25,25,Trairi,BR,-3.2778,-39.2689,82.81,60,14,14.18,few clouds
30,30,Guerrero Negro,MX,27.9769,-114.0611,77.43,58,92,10.29,overcast clouds
31,31,Domoni,KM,-12.2569,44.5319,75.02,67,100,11.30,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                346
City                   346
Country                345
Lat                    346
Lng                    346
Max Temp               346
Humidity               346
Cloudiness             346
Wind Speed             346
Current Description    346
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Georgetown,MY,83.89,few clouds,5.4112,100.3354,
6,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,
8,Rikitea,PF,70.50,broken clouds,-23.1203,-134.9692,
10,Roald,NO,72.00,overcast clouds,62.5833,6.1333,
12,Lompoc,US,82.02,scattered clouds,34.6391,-120.4579,
16,Hithadhoo,MV,80.92,broken clouds,-0.6000,73.0833,
17,Saint-Francois,FR,71.04,clear sky,46.4154,3.9054,
25,Trairi,BR,82.81,few clouds,-3.2778,-39.2689,
30,Guerrero Negro,MX,77.43,overcast clouds,27.9769,-114.0611,
31,Domoni,KM,75.02,overcast clouds,-12.2569,44.5319,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!=""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Georgetown,MY,83.89,few clouds,5.4112,100.3354,Cititel Penang
6,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
8,Rikitea,PF,70.50,broken clouds,-23.1203,-134.9692,People ThankYou
10,Roald,NO,72.00,overcast clouds,62.5833,6.1333,Flem Cabins
12,Lompoc,US,82.02,scattered clouds,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
...,...,...,...,...,...,...,...
660,Meridian,US,86.32,smoke,43.6121,-116.3915,Courtyard by Marriott Boise West/Meridian
662,Kaltenkirchen,DE,76.68,few clouds,53.8333,9.9667,Hotel Dreiklang Business & Spa Resort
664,Sabya,SA,80.22,overcast clouds,17.1495,42.6254,Aseel Hotel Apartment
667,Castres,FR,76.68,clear sky,43.5833,2.1667,Le 81 Hôtel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))